In [1]:
import pandas as pd
from datetime import date, datetime

In [2]:
sales = pd.read_excel('PQ-288 Sales from different markets.xlsx', usecols='A:F', header = 1, nrows = 100)
sales['Product'] = sales['Product'].apply(
    lambda x: ''.join(
        x.lower().split(' ')
    ).title()
)
sales.head()

,Product,Country,Date,Currency,Unit_Price,Quantity
0,Betatab10,Poland,2024-04-15,PLN,2500,3
1,Deltapods,Poland,2024-04-25,PLN,800,3
2,Betatab10,Germany,2024-04-27,EUR,550,5
3,Gammabookpro,Poland,2024-03-31,PLN,7500,2
4,Deltapods,Germany,2024-04-19,EUR,190,5


In [3]:
xRates = pd.read_excel('PQ-288 Sales from different markets.xlsx', usecols='H:K')
xRates = xRates.melt(
    id_vars = 'Date', 
    value_vars = ['EUR', 'PLN', 'USD'], 
    var_name = 'Currency', 
    value_name = 'Rate'
)
xRates.head()

,Date,Currency,Rate
0,2024-03-01,EUR,4.5250
1,2024-03-02,EUR,4.5250
2,2024-03-03,EUR,4.5250
3,2024-03-04,EUR,4.5186
4,2024-03-05,EUR,4.5265


In [4]:
merge = (
    sales
    .merge(xRates, how = 'left', on = ['Currency', 'Date'])
    .assign(Sales = lambda x: x['Quantity'] * x['Unit_Price'] * x['Rate'])
    .assign(Date = lambda x: pd.to_datetime(x['Date']).dt.to_period('M').dt.to_timestamp())
)
merge.head()

,Product,Country,Date,Currency,Unit_Price,Quantity,Rate,Sales
0,Betatab10,Poland,2024-04-01,PLN,2500,3,1.0000,7500.00
1,Deltapods,Poland,2024-04-01,PLN,800,3,1.0000,2400.00
2,Betatab10,Germany,2024-04-01,EUR,550,5,4.5274,12450.35
3,Gammabookpro,Poland,2024-03-01,PLN,7500,2,1.0000,15000.00
4,Deltapods,Germany,2024-04-01,EUR,190,5,4.5078,4282.41


In [5]:
group = merge.groupby(['Country', 'Product', 'Date'])['Sales'].sum().reset_index()

pivot = (
    group
    .pivot(index=['Country', 'Product'], columns='Date', values='Sales')
    .reset_index()
)

pivot.columns.name = None
new_col_names = [
    c.strftime('%b') if isinstance(c, (date, datetime, pd.Timestamp)) else c
    for c in pivot.columns
]
pivot.columns = new_col_names
pivot

,Country,Product,Mar,Apr
0,Germany,Alphaphonex,89518.4400,79422.6400
1,Germany,Betatab10,17401.1750,62207.9700
2,Germany,Deltapods,9443.3800,12870.2770
3,Germany,Gammabookpro,122384.5200,89490.6000
4,Poland,Alphaphonex,35000.0000,55000.0000
5,Poland,Betatab10,27500.0000,52500.0000
6,Poland,Deltapods,21600.0000,3200.0000
7,Poland,Gammabookpro,112500.0000,157500.0000
8,USA,Alphaphonex,53916.1000,49915.6000
9,USA,Betatab10,39342.2079,14521.0996
